## Day 46 Lecture 2 Assignment

In this assignment, we will calculate a similarity matrix using responses to a survey about student life at a university. We will perform clustering on this dataset later on.

In [1]:
!pip install gower

  Created wheel for gower: filename=gower-0.0.5-cp36-none-any.whl size=4232 sha256=4c6932c1d6dc67bbcbe0791e82f5636180109df2a1602524ab387f20bc90a988
  Stored in directory: /root/.cache/pip/wheels/c0/09/9b/072d54d6ced0f43a179852e3f09532d0131e25ff7cb4e5ee75
Successfully built gower


In [2]:
%matplotlib inline

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy.spatial.distance import pdist, squareform
import gower

This dataset consists of 35 binary features, each corresponding to a yes/no question that characterizes the student taking the survey.

This dataset contains a large number of features, each corresponding to a survey question. The feature name summarizes the survey question, so we will not list them all out here.

Load the dataset.

In [3]:
# read in dataset
df = pd.read_csv('https://tf-assets-prod.s3.amazonaws.com/tf-curric/data-science/Data%20Sets%20Clustering/student_life_survey.csv')

For our analysis, we will focus on a specific subset of the survey that is focused on stress. These questions all begin with the string 'Q5'. Filter the columns that meet this criteria (should be 10 in total).

In [4]:
# subset dataframe for just Q5
data = df.loc[:, df.columns.str.contains('Q5')]
data.tail(100)

,Q5-Stressed about Adjustment issues,Q5-Stressed about Academic issues,Q5-Stressed about Financial issues,Q5-Stressed about Family issues,Q5-Stressed about Friendships,Q5-Stressed about Romantic relationships,Q5-Stressed about Health related issues,Q5-Stressed about Career related issues,"Q5-Stressed about My involvement in hostel, clubs, societies, interest groups, etc.",Q5-Stressed about Others
2858,1,1,0,0,1,1,0,0,0,0
2859,1,1,1,1,1,0,1,1,1,0
2860,1,1,1,1,0,1,0,1,1,0
2861,0,1,0,0,1,0,0,1,0,0
2862,1,0,0,0,1,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...
2953,0,1,1,0,0,1,0,0,0,0
2954,0,1,0,0,0,0,0,0,0,0
2955,0,1,0,0,0,0,0,0,0,0
2956,0,1,0,0,0,0,1,1,0,0


We'll assume for the next step that a pair of negative values (i.e. both responses are "no") is as informative as a pair of positive values. Compute the full distance/dissimilarity matrix for the survey data using matching/hamming distance.

The pdist function is a simple way to accomplish this goal, and the squareform function will transform the output of the pdist function into a tabular format that can be convert to a dataframe more easily.

In [5]:
# answer goes here
dist = pd.DataFrame(squareform(pdist(data, metric='hamming')))
dist.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,2918,2919,2920,2921,2922,2923,2924,2925,2926,2927,2928,2929,2930,2931,2932,2933,2934,2935,2936,2937,2938,2939,2940,2941,2942,2943,2944,2945,2946,2947,2948,2949,2950,2951,2952,2953,2954,2955,2956,2957
0,0.0,0.0,0.1,0.6,0.3,0.1,0.2,0.3,0.4,0.4,0.3,0.3,0.5,0.4,0.2,0.3,0.3,0.4,0.4,0.3,0.3,0.2,0.3,0.0,0.3,0.2,0.3,0.2,0.3,0.1,0.3,0.3,0.5,0.1,0.3,0.1,0.1,0.3,0.3,0.0,...,0.2,0.2,0.2,0.1,0.0,0.4,0.1,0.2,0.1,0.0,0.4,0.0,0.1,0.2,0.1,0.2,0.2,0.1,0.0,0.1,0.3,0.1,0.0,0.4,0.0,0.1,0.2,0.1,0.2,0.2,0.1,0.3,0.2,0.3,0.3,0.2,0.0,0.0,0.2,0.2
1,0.0,0.0,0.1,0.6,0.3,0.1,0.2,0.3,0.4,0.4,0.3,0.3,0.5,0.4,0.2,0.3,0.3,0.4,0.4,0.3,0.3,0.2,0.3,0.0,0.3,0.2,0.3,0.2,0.3,0.1,0.3,0.3,0.5,0.1,0.3,0.1,0.1,0.3,0.3,0.0,...,0.2,0.2,0.2,0.1,0.0,0.4,0.1,0.2,0.1,0.0,0.4,0.0,0.1,0.2,0.1,0.2,0.2,0.1,0.0,0.1,0.3,0.1,0.0,0.4,0.0,0.1,0.2,0.1,0.2,0.2,0.1,0.3,0.2,0.3,0.3,0.2,0.0,0.0,0.2,0.2
2,0.1,0.1,0.0,0.7,0.4,0.2,0.1,0.2,0.5,0.3,0.2,0.2,0.4,0.5,0.3,0.2,0.2,0.3,0.3,0.4,0.2,0.3,0.2,0.1,0.2,0.3,0.2,0.1,0.4,0.0,0.2,0.2,0.4,0.2,0.4,0.0,0.2,0.2,0.4,0.1,...,0.3,0.1,0.1,0.0,0.1,0.3,0.0,0.1,0.2,0.1,0.3,0.1,0.2,0.3,0.2,0.1,0.1,0.0,0.1,0.0,0.2,0.2,0.1,0.3,0.1,0.2,0.3,0.0,0.3,0.1,0.2,0.4,0.3,0.4,0.4,0.3,0.1,0.1,0.1,0.1
3,0.6,0.6,0.7,0.0,0.5,0.5,0.8,0.7,0.2,0.6,0.7,0.5,0.5,0.4,0.6,0.7,0.7,0.6,0.6,0.7,0.7,0.6,0.7,0.6,0.5,0.6,0.7,0.8,0.5,0.7,0.7,0.7,0.7,0.5,0.5,0.7,0.7,0.7,0.7,0.6,...,0.6,0.6,0.8,0.7,0.6,0.4,0.7,0.6,0.7,0.6,0.4,0.6,0.5,0.6,0.5,0.8,0.6,0.7,0.6,0.7,0.5,0.5,0.6,0.6,0.6,0.7,0.6,0.7,0.4,0.6,0.7,0.5,0.4,0.5,0.3,0.6,0.6,0.6,0.6,0.8
4,0.3,0.3,0.4,0.5,0.0,0.2,0.3,0.6,0.3,0.3,0.4,0.4,0.4,0.5,0.5,0.6,0.4,0.5,0.3,0.4,0.2,0.3,0.4,0.3,0.4,0.3,0.2,0.3,0.4,0.4,0.4,0.2,0.4,0.4,0.4,0.4,0.2,0.4,0.4,0.3,...,0.3,0.3,0.3,0.4,0.3,0.3,0.4,0.5,0.2,0.3,0.3,0.3,0.2,0.3,0.4,0.5,0.5,0.4,0.3,0.4,0.4,0.4,0.3,0.5,0.3,0.2,0.3,0.4,0.3,0.5,0.2,0.2,0.3,0.4,0.2,0.3,0.3,0.3,0.5,0.3


Print the answers to the following three questions:

1. How many pairs of students are there in our dataset? (Remember to ignore a student pairing with themselves, and avoid double-counting - '3-5' is the same as '5-3'.) 
2. Of these pairs, how many gave identical answers? 
3. How many pairs had no answers in common?

Tip: it may help to first convert the NxN similarity matrix into a table with three columns: response_id_1, response_id_2, and the similarity between them. The new table should have NxN rows and 3 columns.

In [6]:
# answer goes here
dist['id'] = dist.index
sims = dist.melt(id_vars='id')
sims = sims.loc[sims['id'] != sims['variable']]

In [9]:
# I thought this would drop duplicate pairs but it doesn't seem to...
# tried several other methods but didn't find anything that worked
sims = sims.drop_duplicates(subset=['id', 'variable'])

In [13]:
# eliminate pair with self
sims = sims.loc[sims['id'] != sims['variable']]

# rename columns
sims['response_id_1'] = sims['id']
sims['response_id_2'] = sims['variable']

# remove old names
sims.drop(['id', 'variable'], axis=1, inplace=True)

In [14]:

pairs = sims.shape[0]
same = sims[sims['value'] == 0]
dif = sims[sims['value'] == 1]

print('Number of pairs:', pairs)
print('Pairs of identical answers:', same.shape[0])
print('Pairs with no answers in common', dif.shape[0])

Number of pairs: 8746806
Pairs of identical answers: 273464
Pairs with no answers in common 582


Without solving for double counting, there are 2958 pairs. 

For each student, calculate their average similarity with all other students. Then print the following:

1. On average, which student (i.e. which response_id) had the most similar responses to the survey population as a whole? What was the average similarity for this student, and what were their answers?
2. Repeat part 1, but for the student with the least similar responses to the survey population as a whole.
3. What do we notice about these students? What broader inferences can we make about the student population as a whole?

In [173]:
# group by response_id_1 and find mean for value column
avg_sims = sims.groupby(by='response_id_1').mean()
avg_sims.sort_values(by='value', ascending=True, inplace=True)
most_sim = avg_sims.index[0]

print('Student {} had the most similar response with an average score of {}'.format(most_sim, round(avg_sims.value[most_sim], 2)))
print('The student\'s scores were: {}'.format(data.loc[most_sim].values))

Student 0 had the most similar response with an average score of 0.22
The student's scores were: [0 1 0 0 0 0 0 0 0 0]


In [172]:
print(avg_sims.head())
avg_sims.value[440]

                  value
response_id_1          
0              0.224822
440            0.224822
1865           0.224822
1869           0.224822
433            0.224822


0.2248224551910744

In [175]:
# group by response_id_1 and find mean for value column
avg_sims = sims.groupby(by='response_id_1').mean()
avg_sims.sort_values(by='value', ascending=False, inplace=True)
most_dif = avg_sims.index[0]

print('Student {} had the most different response with an average score of {}'.format(most_dif, round(avg_sims.value[most_dif], 2)))
print('The student\'s scores were: {}'.format(data.loc[most_dif].values))

Student 2899 had the most different response with an average score of 0.62
The student's scores were: [1 1 1 1 1 0 1 1 1 1]


In [187]:
data.mean()

Q5-Stressed about Adjustment issues                                                    0.306626
Q5-Stressed about Academic issues                                                      0.922245
Q5-Stressed about Financial issues                                                     0.386072
Q5-Stressed about Family issues                                                        0.147397
Q5-Stressed about Friendships                                                          0.269777
Q5-Stressed about Romantic relationships                                               0.165652
Q5-Stressed about  Health related issues                                               0.151116
Q5-Stressed about Career related issues                                                0.455037
Q5-Stressed about My involvement in hostel, clubs, societies, interest groups, etc.    0.274172
Q5-Stressed about Others                                                               0.013861
dtype: float64

We know that the most similar student is significantly less stressed than the most different student. Based on the most similar student's responses, we can assume that most students are not very stressed. Looking at the means for each column, we can see that this is true. We can also see that the area that is most stressful for students if academics. The most similar student is stressed in this area. 